<a href="https://colab.research.google.com/github/ankitgoelcmu/DeepLearning/blob/main/Fine_Tuning_Base_Model_PEFT_(QLORA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this Notebook - I will be doing threat detection using the Mireu-Lab/NSL-KDD dataset (a benchmark for network intrusion detection with binary classes: Normal, Anamoly)
### This guide fine-tunes DistilBERT (lightweight text model) by converting tabular features to log-like strings (e.g., "duration=0 protocol=tcp src_bytes=215"), optimizes with QLoRA (4-bit), quantizes/prunes for speed.

## I will be using Hugging face for the dataset + base model + PEFT.

## TODO Deploy to Gradio

##Hugging Face Cheat Sheet (Quick Refresher)

1. Transformers: Models/tokenizers (e.g., AutoModelForSequenceClassification).
2. Datasets: load_dataset('enron_spam')—easy data.
3. PEFT/LoRA: LoraConfig(r=8)—adapters for efficient FT.
4. BitsAndBytes: BitsAndBytesConfig(load_in_4bit=True)—quantization.
5. Trainer: Trainer(model, args, dataset)—handles loops/metrics.
6. Gradio: gr.Interface(fn=predict, ...).launch()—UI demo.




#Step 1: Loading dataset and exploring its features, labels, class etc

In [ ]:
from datasets import load_dataset
train_dataset = load_dataset("Mireu-Lab/NSL-KDD", split="train")
test_dataset  = load_dataset("Mireu-Lab/NSL-KDD", split="test")

# Lets select 2000 random dataset values by shuffling the dataset and then selecting the first 200 elements.
train_dataset = train_dataset.shuffle().select(range(4000))
test_dataset = test_dataset.shuffle().select(range(200))

print(f"train_dataset size: {len(train_dataset)}")
print(f"test_dataset size: {len(test_dataset)}")
print("First 5 samples from the randomly selected 200:")
print(train_dataset[:5])

In [ ]:
#Inspect the first dataset
train_dataset[0]

In [ ]:
train_dataset.column_names

In [ ]:
random_integers = random.sample(range(len(train_dataset)), 5)
train_dataset[random_integers]

In [ ]:
train_dataset.unique("class")

In [ ]:
#Checking count of  labels
from collections import Counter

Counter(train_dataset["class"])

In [ ]:
# Turn our dataset into a DataFrame and get a random sample
random_integers = random.sample(range(len(train_dataset)), 10)
network_traffic = pd.DataFrame(train_dataset[random_integers])
network_traffic

In [ ]:
network_traffic["class"].value_counts()

# Creating a mapping from Class/Labels to numbers [0 -> Normal, 1 -> Anomaly]

In [ ]:
# Create mapping from id2label and label2id
id2label = {0: "normal", 1: "anomaly"}
label2id = {"normal": 0, "anomaly": 1}

print(f"Label to ID mapping: {label2id}")
print(f"ID to Label mapping: {id2label}")


#Now turn our Class/Labels in 0 or 1

def map_class_to_id(example):
    example["class"] = label2id[example["class"]]
    return example

#Now Test this function with our train_dataset[0]
map_class_to_id(train_dataset[0])

In [ ]:
#Now lets convert Class to IDs for or whole data set - train + test
train_dataset = train_dataset.map(map_class_to_id)
test_dataset  = test_dataset.map(map_class_to_id)

train_dataset.features["class"]

In [ ]:
train_dataset[0]

#Preprocess: Convert Tablular Features to Text Strings and store the String in "text" Column. This will alllow us to tokenize the "text"
`(e.g = "duration=0 protocol=tcp ..."`

In [ ]:
def tabular_to_text(examples):
  #Tabular example is dict with keys like 'duration', 'protocol_type', 'class'
  texts = []
  for i in range(len(examples["duration"])):
      text = (
            f"duration={examples['duration'][i]} "
            f"protocol_type={examples['protocol_type'][i]} "
            f"service={examples['service'][i]} "
            f"flag={examples['flag'][i]} "
            f"src_bytes={examples['src_bytes'][i]} "
            f"dst_bytes={examples['dst_bytes'][i]} "
            f"land={examples['land'][i]} "
            f"wrong_fragment={examples['wrong_fragment'][i]} "
            f"urgent={examples['urgent'][i]} "
            f"hot={examples['hot'][i]} "
            f"num_failed_logins={examples['num_failed_logins'][i]} "
            f"logged_in={examples['logged_in'][i]} "
            f"num_compromised={examples['num_compromised'][i]} "
            f"root_shell={examples['root_shell'][i]} "
            f"su_attempted={examples['su_attempted'][i]} "
            f"num_root={examples['num_root'][i]} "
            f"num_file_creations={examples['num_file_creations'][i]} "
            f"num_shells={examples['num_shells'][i]} "
            f"num_access_files={examples['num_access_files'][i]} "
            f"num_outbound_cmds={examples['num_outbound_cmds'][i]} "
            f"is_host_login={examples['is_host_login'][i]} "
            f"is_guest_login={examples['is_guest_login'][i]} "
            f"count={examples['count'][i]} "
            f"srv_count={examples['srv_count'][i]} "
            f"serror_rate={examples['serror_rate'][i]} "
            f"srv_serror_rate={examples['srv_serror_rate'][i]} "
            f"rerror_rate={examples['rerror_rate'][i]} "
            f"srv_rerror_rate={examples['srv_rerror_rate'][i]} "
            f"same_srv_rate={examples['same_srv_rate'][i]} "
            f"diff_srv_rate={examples['diff_srv_rate'][i]} "
            f"srv_diff_host_rate={examples['srv_diff_host_rate'][i]} "
            f"dst_host_count={examples['dst_host_count'][i]} "
            f"dst_host_srv_count={examples['dst_host_srv_count'][i]} "
            f"dst_host_same_srv_rate={examples['dst_host_same_srv_rate'][i]} "
            f"dst_host_diff_srv_rate={examples['dst_host_diff_srv_rate'][i]} "
            f"dst_host_same_src_port_rate={examples['dst_host_same_src_port_rate'][i]} "
            f"dst_host_srv_diff_host_rate={examples['dst_host_srv_diff_host_rate'][i]} "
            f"dst_host_serror_rate={examples['dst_host_serror_rate'][i]} "
            f"dst_host_srv_serror_rate={examples['dst_host_srv_serror_rate'][i]} "
            f"dst_host_rerror_rate={examples['dst_host_rerror_rate'][i]} "
            f"dst_host_srv_rerror_rate={examples['dst_host_srv_rerror_rate'][i]} "
        )
      texts.append(text)
  return {'text': texts}

In [ ]:
#Apply to dataset
train_dataset = train_dataset.map(tabular_to_text, batched=True)
test_dataset = test_dataset.map(tabular_to_text, batched=True)

#Lets check the newly inserted "text"
train_dataset[0]["text"]

In [ ]:
train_dataset[0]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
tokenizer

In [ ]:
tokenizer.vocab_size
tokenizer("I am Ankit Goel")

In [ ]:
tokenizer.model_max_length

#Tokenize the text logs

In [ ]:
def tokenize_text(examples):
  return tokenizer(examples["text"],
                   padding=True,
                   truncation=True)


# Map our tokenize_text function to the dataset, remove all the columns except 'class'
# Final columns = 'input_ids', 'attention_mask'(from tokenize) + 'class'
tokenized_train_dataset = train_dataset.map(tokenize_text, batched=True, batch_size=1000,
                                            remove_columns=[col for col in train_dataset.column_names if col != 'class'])
tokenized_test_dataset = test_dataset.map(tokenize_text, batched=True, batch_size=1000,
                                            remove_columns=[col for col in train_dataset.column_names if col != 'class'])




In [ ]:
tokenized_train_dataset


In [ ]:
# Get two samples from the tokenized dataset
train_tokenized_sample = tokenized_train_dataset[0]
test_tokenized_sample = tokenized_test_dataset[0]

for key in train_tokenized_sample.keys():
    print(f"[INFO] Key: {key}")
    print(f"Train sample: {train_tokenized_sample[key]}")
    print(f"Test sample: {test_tokenized_sample[key]}")
    print("")

In [ ]:
#Now time to format Train + Test dataset for PyTorch compatibility
tokenized_test_dataset = tokenized_test_dataset.rename_column('class', 'labels')
tokenized_train_dataset = tokenized_train_dataset.rename_column('class', 'labels')
tokenized_test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# Verify (label preserved!)
print(f"Train columns: {tokenized_test_dataset.column_names}")  # ['input_ids', 'attention_mask', 'class']
print(f"Sample label: {tokenized_test_dataset[0]['labels']}")  # e.g., 0 (normal)
print(f"Sample input_ids len: {len(tokenized_test_dataset[0]['input_ids'])}")

#Setting up an evaluation metric

In [ ]:
import evaluate
import numpy as np
from typing import Tuple

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1") # Load the f1 metric

def compute_accuracy(predictions_and_labels: Tuple[np.array, np.array]):
  """
  Computes the accuracy and F1-score of a model by comparing the predictions and labels.
  """
  predictions, labels = predictions_and_labels

  # Get highest prediction probability of each prediction if predictions are probabilities
  if len(predictions.shape) >= 2:
    predictions = np.argmax(predictions, axis=1)

  accuracy_result = accuracy_metric.compute(predictions=predictions, references=labels)
  f1_result = f1_metric.compute(predictions=predictions, references=labels, average="binary") # Calculate F1 for binary classification

  return {**accuracy_result, **f1_result} # Return both metrics

In [ ]:
#lets test it out
# Create example list of predictions and labels
example_predictions_all_correct = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
example_predictions_one_wrong = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
example_labels = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Test the function
print(f"Accuracy when all predictions are correct: {compute_accuracy((example_predictions_all_correct, example_labels))}")
print(f"Accuracy when one prediction is wrong: {compute_accuracy((example_predictions_one_wrong, example_labels))}")

In [ ]:
def count_params(model):
    """
    Count the parameters of a PyTorch model.
    """
    trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_parameters = sum(p.numel() for p in model.parameters())

    return {"trainable_parameters": trainable_parameters, "total_parameters": total_parameters}

# Count the parameters of the model
#count_params(model)



#Creating Dirs to Save the Model

In [ ]:
# Create model output directory
from pathlib import Path

# Create models directory
models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

# Create model save name
model_save_name = "network-threat_classifier-distilbert-base-uncased"

# Create model save path
model_save_dir = Path(models_dir, model_save_name)

model_save_dir

#PEFT TIME!!!



In [ ]:
#!pip install --upgrade transformers bitsandbytes accelerate peft torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

from peft import LoraConfig, get_peft_model

#Define Quatinzed Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Back to 4-bit—saves 75% memory
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model with quantization (same params, now quantized)
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="bert-base-uncased", #distilbert/distilbert-base-uncased
    quantization_config=bnb_config,
    num_labels=2, # can customize this to the number of classes in your dataset
    id2label=id2label, # mappings from class IDs to the class labels (for classification tasks)
    label2id=label2id
)


config = LoraConfig(
    task_type="SEQ_CLS", # Change task_type for sequence classification
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value"], # Corrected target modules for DistilBert
    lora_dropout=0.1,
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

#Inspect the output of `model.print_trainable_parameters()`

##trainable params: 443,906 || all params: 109,927,684 || trainable%: 0.4038

##Note:

1.   Total Params: ~109M (BERT-base).
2.   Trainable: ~444k (0.4%)—only LoRA adapters + classifier head..  



In [ ]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available! Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")

In [ ]:
model

#TODO : Add text to explore above model

# Define Training Arguments with TrainingArguments

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments (
    output_dir=model_save_dir,
    learning_rate=2e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    use_cpu=False,
    push_to_hub=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    #class_weight='balanced',
    logging_steps=1, # Log every step to see the training loss for small dataset
    report_to=[] # Explicitly disable external reporting to ensure console output
)

In [ ]:
training_args

# Setting up an instance of Trainer


In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_accuracy)

In [ ]:
#Lets TRAIN TRAIN TRAIN

results = trainer.train()

In [ ]:
qlora_results = trainer.evaluate()
qlora_acc = qlora_results['eval_accuracy']
qlora_f1 = qlora_results['eval_f1'] # This will now be available
print(f"QLoRA Acc: {qlora_acc:.3f}, F1: {qlora_f1:.3f}")

In [ ]:
# Inspect training metrics
for key, value in results.metrics.items():
    print(f"{key}: {value}")

In [ ]:
#Save the Model
print(f"[INFO] Saving model to {model_save_dir}")
trainer.save_model(output_dir=model_save_dir)

In [ ]:
# Get training history
trainer_history_all = trainer.state.log_history
trainer_history_all[:4]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Extract training and evaluation loss from trainer_history_all
train_losses = []
eval_losses = []
epochs = []

for log_entry in trainer_history_all:
    if 'loss' in log_entry: # Training loss
        train_losses.append(log_entry['loss'])
        epochs.append(log_entry['epoch'])
    if 'eval_loss' in log_entry: # Evaluation loss
        eval_losses.append(log_entry['eval_loss'])

# Adjust epochs for eval_losses to match the number of eval steps, not all training steps
# The number of eval_losses should correspond to the number of eval_steps, which are per epoch.
# We can use a simpler approach by aligning them based on when they appear in the history.

# Filter for entries that have 'loss' (training loss) and 'eval_loss' (validation loss)
training_logs = [entry for entry in trainer_history_all if 'loss' in entry and 'learning_rate' in entry]
evaluation_logs = [entry for entry in trainer_history_all if 'eval_loss' in entry]

train_epochs = [entry['epoch'] for entry in training_logs]
train_losses = [entry['loss'] for entry in training_logs]

eval_epochs = [entry['epoch'] for entry in evaluation_logs]
eval_losses = [entry['eval_loss'] for entry in evaluation_logs]

# Plotting the loss curves
plt.figure(figsize=(10, 6))
sns.lineplot(x=train_epochs, y=train_losses, label='Training Loss')
sns.lineplot(x=eval_epochs, y=eval_losses, label='Validation Loss')

plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()